## Lab - TensorRT + Profiling - YOLOv4-Tiny TensorRT
## E6692 Spring 2022


In this notebook we outline the steps for converting your trained Darknet YOLOv4-Tiny model from the previous lab (Lab-YOLOv4-Tiny) to a TensorRT model. We use the scripts **utils/yolo2onnx.py** to first convert the Darknet model to ONNX. ONNX stands for the Open Neural Network Exchange, and is a standardized format for converting machine learning and deep learning models between frameworks. Once you have an ONNX version of YOLOv4-Tiny, we use the script **utils/onnx2tensorrt.py** to convert from ONNX to TensorRT automatically using the ONNX parser. 

In [ ]:
# TODO: install onnx 1.4.1

!pip3 install onnx==1.4.1

**TODO:** Use the command `python3 yolo2onnx.py <cfg_path> <weights_path> <onnx_model_path>` to generate an ONNX graph from the darknet YOLOv4-Tiny model you trained in the previous lab. `cfg_path` specifies the Darknet configuration file path, `weights_path` specifies the Darknet weights file path, and `onnx_model_path` specifies the path where the ONNX graph will be saved.

**TODO:** Navigate to the **plugins** directory and enter the command `make` to compile the **yolo_layer** plugin code.

### Why did we have to do that? What is yolo_layer?

**TODO:** Your answer here.

**TODO:** Use the command `python3 onnx2tensorrt.py -cfg <cfg_path> -o <onnx_path> -t <trt_model_path>` to generate a serialized TensorRT engine for YOLOv4-Tiny. `cfg_path` is the path to the Darknet model configuration file, `onnx_path` is the path to the onnx model graph, and `trt_model_path` is the path that the serialized TensorRT engine will be written to.

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

from utils.yoloTRT import TrtYOLO
from utils.utils import draw_bboxes

trt_engine = './engines/yolov4-tiny-person-vehicle.trt'
test_frames = './test_frames'

CONF_THRESH = 0.4

%load_ext autoreload
%autoreload 2

**TODO:** Load the YOLOv4-Tiny TensorRT engine with **TrtYOLO()** in **yoloTRT.py**. This class has all the functions needed for inference with the TensorRT version of YOLO.

In [ ]:
# TODO: load the TRT YOLOv4-Tiny with TrtYOLO()


**TODO:** In **test_frames** there are several images taken from the Mudd building cameras and other intersections in NYC. Use the TensorRT YOLOv4-Tiny model to perform inference on all the test frames. Do the inference and display the detected images below:

In [ ]:
# TODO: do inference with TRT YOLOv4-Tiny and display the detections


## Comment on the effectiveness of the detection. How does the model generalize to new traffic intersection domains? (1st floor, 2nd floor, 12th floor, and other intersection scenes)

**TODO:** Your answer here.
